## Scott Berry - Final Project Chatbot

Install Chatterbot and spaCy libraries as well as other dependencies

In [17]:
## !pip install chatterbot==1.0.4
## !pip install chatterbot_corpus
## !pip install pytz
## !python -m spacy download en_core_web_sm
## modify time.clock -> time.time in ~\AppData\Roaming\Python\Python310\site-packages\sqlalchemy\util\compat.py

Initialize chatbot

In [18]:
from chatterbot import ChatBot

bot = ChatBot(
    "Scorberr_Chatbot",
    logic_adapters=[
        {
            "import_path": "chatterbot.logic.BestMatch",
            "default_response": "Sorry, could you rephrase that? I didn't understand.",
            "maximum_similarity_threshold": 0.90
        }
    ],
    preprocessors=[
        "chatterbot.preprocessors.clean_whitespace",
        "chatterbot.preprocessors.unescape_html",
        "chatterbot.preprocessors.convert_to_ascii"
    ]
)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\berry\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\berry\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\berry\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Train chatbot

In [19]:
from chatterbot.trainers import ListTrainer
trainer = ListTrainer(bot)

trainer.train([
    "Hi, how can I assist?",
    "Hello, what temperature does water boil at?",
    "Water boils at 212 degrees Fahrenheit and 100 degrees Celsius.",
    "Okay, thank you!",
    "You're welcome, have a good day!"
])

List Trainer: [####################] 100%


Test Chatbot

In [20]:
response = bot.get_response("water boiling temperature")
print(response)

Sorry, could you rephrase that? I didn't understand.


Create a while loop for chatbot activity

In [21]:
name = input("Enter your name: ")

print("Welcome to Scorberr\'s chatbot, how can I help?")

while True:

    request = input(name + ":")

    if request.lower() == "bye":
        print("Bot: Bye")
        break
    else:
        response = bot.get_response(request)
        print("Bot: ", response)

Welcome to Scorberr's chatbot, how can I help?
Bot:  Hello, what temperature does water boil at?
Bot:  Sorry, could you rephrase that? I didn't understand.
Bot:  Sorry, could you rephrase that? I didn't understand.
Bot: Bye


Train on chatterbot corpus library

In [22]:
from chatterbot.trainers import ChatterBotCorpusTrainer

trainer = ChatterBotCorpusTrainer(bot)

corpus_library = [
    "english", "AI", "botprofile", "conversations", "greetings", "health", "science"
]
for corpus in corpus_library:
    trainer.train("chatterbot.corpus." + corpus)

AttributeError: module 'collections' has no attribute 'Hashable'